# 🔧 고급 도메인 특성 공학

## 목표: "왜 기업이 부도가 나는가?"에 대한 도메인 지식을 특성으로 구현

이 노트북에서는 단순한 통계적 특성이 아닌, 부도의 근본 원인을 포착하는 도메인 특화 특성을 생성합니다.

## 1. 환경 설정 및 데이터 로딩

In [6]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# 한글 폰트 설정
import platform
if platform.system() == 'Darwin':
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
else:
    plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

# 데이터 로딩
df = pd.read_csv('../data/기업신용평가정보_210801.csv', encoding='utf-8')
target_col = '모형개발용Performance(향후1년내부도여부)'

print(f"✅ 데이터 로딩 완료: {df.shape[0]:,} 기업, {df.shape[1]:,} 변수")
print(f"✅ 부도율: {df[target_col].mean()*100:.2f}%")

✅ 데이터 로딩 완료: 50,105 기업, 159 변수
✅ 부도율: 1.51%


## 2. 유동성 위기 조기 감지 특성

In [7]:
def create_liquidity_crisis_features(df):
    """유동성 위기를 조기에 감지하는 특성 생성"""
    
    features = pd.DataFrame(index=df.index)
    
    # 1. 즉각적 지급능력
    if '현금' in df.columns and '유동부채' in df.columns:
        features['즉각지급능력'] = (df['현금'] + df.get('현금성자산', 0)) / (df['유동부채'] + 1)
        features['현금소진일수'] = (df['현금'] + df.get('현금성자산', 0)) / (df.get('영업비용', df['매출원가']) / 365 + 1)
    
    # 2. 운전자본 건전성
    if '유동자산' in df.columns and '유동부채' in df.columns:
        features['운전자본'] = df['유동자산'] - df['유동부채']
        features['운전자본비율'] = features['운전자본'] / (df.get('매출액', 1) + 1)
        features['운전자본_대_자산'] = features['운전자본'] / (df.get('자산총계', 1) + 1)
    
    # 3. 긴급 자금조달 여력
    if '매출채권' in df.columns and '단기차입금' in df.columns:
        features['긴급유동성'] = (df['현금'] + df.get('현금성자산', 0) + df['매출채권'] * 0.8) / (df['단기차입금'] + 1)
    
    # 4. 유동성 압박 지표
    if '유동부채' in df.columns and '부채총계' in df.columns:
        features['단기부채비중'] = df['유동부채'] / (df['부채총계'] + 1)
        features['유동성압박지수'] = (df['유동부채'] / (df['유동자산'] + 1)) * (df['부채총계'] / (df['자산총계'] + 1))
    
    # 5. 현금흐름 기반 유동성
    if '영업활동현금흐름' in df.columns:
        features['OCF_대_유동부채'] = df['영업활동현금흐름'] / (df.get('유동부채', 1) + 1)
        features['현금창출능력'] = df['영업활동현금흐름'] / (df.get('매출액', 1) + 1)
    
    print(f"✅ 유동성 위기 특성 {features.shape[1]}개 생성 완료")
    return features

liquidity_features = create_liquidity_crisis_features(df)
print("\n생성된 유동성 특성:")
print(liquidity_features.columns.tolist())

✅ 유동성 위기 특성 10개 생성 완료

생성된 유동성 특성:
['즉각지급능력', '현금소진일수', '운전자본', '운전자본비율', '운전자본_대_자산', '긴급유동성', '단기부채비중', '유동성압박지수', 'OCF_대_유동부채', '현금창출능력']


## 3. 지급불능 패턴 특성

In [8]:
def create_insolvency_features(df):
    """지급불능 위험을 포착하는 특성 생성"""
    
    features = pd.DataFrame(index=df.index)
    
    # 1. 자본 잠식도
    if '자본총계' in df.columns:
        features['자본잠식여부'] = (df['자본총계'] < 0).astype(int)
        features['자본잠식도'] = np.where(df.get('납입자본금', 1) > 0, 
                                       np.maximum(0, 1 - df['자본총계'] / df.get('납입자본금', 1)), 0)
    
    # 2. 차입금 의존도
    if '단기차입금' in df.columns and '장기차입금' in df.columns:
        features['총차입금'] = df['단기차입금'] + df['장기차입금']
        features['차입금의존도'] = features['총차입금'] / (df.get('자산총계', 1) + 1)
        features['차입금_대_매출'] = features['총차입금'] / (df.get('매출액', 1) + 1)
    
    # 3. 이자보상능력
    if '영업손익' in df.columns and '금융비용' in df.columns:
        features['이자보상배율'] = (df['영업손익'] + df.get('감가상각비', 0)) / (df['금융비용'] + 1)
        features['이자부담률'] = df['금융비용'] / (df.get('매출액', 1) + 1)
    
    # 4. 부채 상환 능력
    if '당기순이익' in df.columns and '부채총계' in df.columns:
        features['부채상환년수'] = df['부채총계'] / (df['당기순이익'] + df.get('감가상각비', 0) + 1)
        features['순부채비율'] = (df['부채총계'] - df.get('현금', 0)) / (df.get('자본총계', 1) + 1)
    
    # 5. 레버리지 위험
    if '자산총계' in df.columns and '자본총계' in df.columns:
        features['재무레버리지'] = df['자산총계'] / (df['자본총계'].abs() + 1)
        features['부채레버리지'] = df.get('부채총계', 0) / (df['자본총계'].abs() + 1)
    
    print(f"✅ 지급불능 패턴 특성 {features.shape[1]}개 생성 완료")
    return features

insolvency_features = create_insolvency_features(df)
print("\n생성된 지급불능 특성:")
print(insolvency_features.columns.tolist())

✅ 지급불능 패턴 특성 8개 생성 완료

생성된 지급불능 특성:
['자본잠식여부', '자본잠식도', '이자보상배율', '이자부담률', '부채상환년수', '순부채비율', '재무레버리지', '부채레버리지']


## 4. 재무조작 탐지 특성 (Beneish M-Score 한국형)

In [9]:
def create_manipulation_detection_features(df):
    """재무조작 가능성을 탐지하는 특성 생성"""
    
    features = pd.DataFrame(index=df.index)
    
    # 공통 변수 안전하게 확보 (부채비율이 없으면 계산)
    if '부채비율' in df.columns:
        부채비율 = df['부채비율']
    elif '부채총계' in df.columns and '자본총계' in df.columns:
        부채비율 = df['부채총계'] / (df['자본총계'].abs() + 1) * 100
    else:
        부채비율 = 100  # 기본값 (데이터 부족 시)

    # 1. 매출채권 이상 증가
    if '매출채권' in df.columns and '매출액' in df.columns:
        features['매출채권회전율'] = df['매출액'] / (df['매출채권'] + 1)
        features['매출채권비율'] = df['매출채권'] / (df['매출액'] + 1)
        # 수정된 부분: df['부채비율'] 대신 위에서 확보한 변수 사용
        features['매출채권_이상지표'] = features['매출채권비율'] * (부채비율 / 100)
    
    # 2. 재고자산 이상 적체
    if '재고자산' in df.columns and '매출원가' in df.columns:
        features['재고회전율'] = df['매출원가'] / (df['재고자산'] + 1)
        features['재고보유일수'] = 365 / (features['재고회전율'] + 0.1)
        features['재고_이상지표'] = df['재고자산'] / (df.get('자산총계', 1) + 1) * 100
    
    # 3. 발생액(Accruals) 품질
    if '당기순이익' in df.columns and '영업활동현금흐름' in df.columns:
        features['총발생액'] = df['당기순이익'] - df['영업활동현금흐름']
        features['발생액비율'] = features['총발생액'] / (df.get('자산총계', 1) + 1)
        features['현금흐름품질'] = df['영업활동현금흐름'] / (df['당기순이익'] + 1)
    
    # 4. 비용 자본화 의심
    if '무형자산' in df.columns and '영업비용' in df.columns:
        features['무형자산비율'] = df['무형자산'] / (df.get('자산총계', 1) + 1)
        features['비용자본화지표'] = df['무형자산'] / (df['영업비용'] + 1)
    
    # 5. 매출총이익률 악화
    if '매출총이익' in df.columns and '매출액' in df.columns:
        features['매출총이익률'] = df['매출총이익'] / (df['매출액'] + 1) * 100
        features['영업레버리지'] = df.get('영업손익', 0) / (df['매출총이익'] + 1)
    
    # 6. 판관비 이상 증가
    if '판매비와관리비' in df.columns and '매출액' in df.columns:
        features['판관비율'] = df['판매비와관리비'] / (df['매출액'] + 1) * 100
        features['판관비효율성'] = df.get('영업손익', 0) / (df['판매비와관리비'] + 1)
    
    # 7. M-Score 종합
    m_score = 0
    if '매출채권비율' in features.columns:
        m_score += features['매출채권비율'] * 0.92
    if '재고_이상지표' in features.columns:
        m_score += features['재고_이상지표'] * 0.528
    if '발생액비율' in features.columns:
        m_score += features['발생액비율'] * 4.679
    if '무형자산비율' in features.columns:
        m_score += features['무형자산비율'] * 0.404
    
    features['M_Score_한국형'] = m_score - 2.22  # 한국 시장 조정
    features['재무조작위험'] = (features['M_Score_한국형'] > 0).astype(int)
    
    print(f"✅ 재무조작 탐지 특성 {features.shape[1]}개 생성 완료")
    return features

manipulation_features = create_manipulation_detection_features(df)
print("\n생성된 재무조작 탐지 특성:")
print(manipulation_features.columns.tolist())

✅ 재무조작 탐지 특성 15개 생성 완료

생성된 재무조작 탐지 특성:
['매출채권회전율', '매출채권비율', '매출채권_이상지표', '재고회전율', '재고보유일수', '재고_이상지표', '총발생액', '발생액비율', '현금흐름품질', '매출총이익률', '영업레버리지', '판관비율', '판관비효율성', 'M_Score_한국형', '재무조작위험']


## 5. 한국 시장 특화 특성

In [10]:
def create_korean_market_features(df):
    """한국 시장 특화 리스크 특성 생성"""
    
    features = pd.DataFrame(index=df.index)
    
    # 1. 대기업 의존도 (매출 집중도 프록시)
    if '매출채권' in df.columns and '매출액' in df.columns:
        features['매출채권집중도'] = df['매출채권'] / (df['매출액'] + 1)
        features['거래처리스크'] = features['매출채권집중도'] * df.get('부채비율', 100) / 100
    
    # 2. 수출 민감도 (제조업 특화)
    if '업종(중분류)' in df.columns:
        제조업여부 = df['업종(중분류)'].str.contains('제조', na=False).astype(int)
        features['제조업_레버리지위험'] = 제조업여부 * df.get('부채비율', 0) / 100
        features['제조업_재고위험'] = 제조업여부 * df.get('재고자산', 0) / (df.get('자산총계', 1) + 1)
    
    # 3. 부동산 연계 리스크
    if '유형자산' in df.columns and '자산총계' in df.columns:
        features['유형자산비중'] = df['유형자산'] / (df['자산총계'] + 1)
        features['자산유동성위험'] = features['유형자산비중'] * df.get('부채비율', 100) / 100
    
    # 4. 외감 기업 여부
    if '외감구분' in df.columns:
        features['외감기업'] = (df['외감구분'] == 'Y').astype(int)
        features['비외감_리스크'] = (1 - features['외감기업']) * 2  # 비외감 기업 페널티
    
    # 5. 업력 리스크
    if '설립일자' in df.columns:
        try:
            설립연도 = pd.to_datetime(df['설립일자'], errors='coerce').dt.year
            features['업력'] = 2021 - 설립연도  # 데이터 기준년도 2021
            features['신생기업리스크'] = (features['업력'] < 3).astype(int) * 3
            features['업력_안정성'] = np.log1p(features['업력'])
        except:
            pass
    
    # 6. 규모 리스크
    if '자산총계' in df.columns:
        features['기업규모_로그'] = np.log1p(df['자산총계'])
        # 중소기업 리스크 (자산 100억 미만)
        features['중소기업리스크'] = (df['자산총계'] < 10000000).astype(int)
    
    print(f"✅ 한국 시장 특화 특성 {features.shape[1]}개 생성 완료")
    return features

korean_features = create_korean_market_features(df)
print("\n생성된 한국 시장 특화 특성:")
print(korean_features.columns.tolist())

✅ 한국 시장 특화 특성 13개 생성 완료

생성된 한국 시장 특화 특성:
['매출채권집중도', '거래처리스크', '제조업_레버리지위험', '제조업_재고위험', '유형자산비중', '자산유동성위험', '외감기업', '비외감_리스크', '업력', '신생기업리스크', '업력_안정성', '기업규모_로그', '중소기업리스크']


## 6. 이해관계자 행동 패턴 특성

In [11]:
def create_stakeholder_behavior_features(df):
    """이해관계자 행동 패턴을 포착하는 특성 생성"""
    
    features = pd.DataFrame(index=df.index)
    
    # 1. 신용 행동 패턴
    credit_cols = [col for col in df.columns if '연체' in col]
    if credit_cols:
        features['총연체건수'] = df[credit_cols].sum(axis=1)
        features['연체여부'] = (features['총연체건수'] > 0).astype(int)
        features['연체심각도'] = features['총연체건수'] * df.get('부채비율', 100) / 100
    
    # 2. 세금 체납 리스크
    tax_cols = [col for col in df.columns if '체납' in col or '세금' in col]
    if tax_cols:
        features['세금체납건수'] = df[tax_cols].sum(axis=1)
        features['세금체납리스크'] = (features['세금체납건수'] > 0).astype(int) * 5
    
    # 3. 공공정보 리스크
    public_cols = [col for col in df.columns if any(k in col for k in ['압류', '소송', '공공'])]
    if public_cols:
        features['공공정보리스크'] = df[public_cols].sum(axis=1)
        features['법적리스크'] = (features['공공정보리스크'] > 0).astype(int) * 3
    
    # 4. 신용등급 리스크
    rating_cols = [col for col in df.columns if '신용평가등급' in col]
    if rating_cols:
        # 신용등급을 숫자로 변환 (낮은 등급일수록 높은 점수)
        for col in rating_cols:
            if df[col].dtype == 'object':
                # 등급을 리스크 점수로 매핑
                grade_map = {'AAA': 1, 'AA': 2, 'A': 3, 'BBB': 4, 'BB': 5, 'B': 6, 'CCC': 7, 'CC': 8, 'C': 9, 'D': 10}
                features['신용등급점수'] = df[col].map(grade_map).fillna(10)
            else:
                features['신용등급점수'] = df[col]
            break
    
    # 5. 종합 신뢰도 지표
    features['이해관계자_불신지수'] = (
        features.get('연체여부', 0) * 2 +
        features.get('세금체납리스크', 0) +
        features.get('법적리스크', 0) +
        features.get('신용등급점수', 0) / 2
    )
    
    print(f"✅ 이해관계자 행동 패턴 특성 {features.shape[1]}개 생성 완료")
    return features

stakeholder_features = create_stakeholder_behavior_features(df)
print("\n생성된 이해관계자 행동 특성:")
print(stakeholder_features.columns.tolist())

✅ 이해관계자 행동 패턴 특성 9개 생성 완료

생성된 이해관계자 행동 특성:
['총연체건수', '연체여부', '연체심각도', '세금체납건수', '세금체납리스크', '공공정보리스크', '법적리스크', '신용등급점수', '이해관계자_불신지수']


## 7. 복합 리스크 지표 생성

In [12]:
def create_composite_risk_indices(df, liquidity_f, insolvency_f, manipulation_f, korean_f, stakeholder_f):
    """여러 도메인 특성을 결합한 복합 리스크 지표 생성"""
    
    features = pd.DataFrame(index=df.index)
    
    # 1. 재무 건전성 종합 지수
    financial_health_components = []
    
    if '즉각지급능력' in liquidity_f.columns:
        financial_health_components.append(liquidity_f['즉각지급능력'].fillna(0))
    if '이자보상배율' in insolvency_f.columns:
        financial_health_components.append(np.clip(insolvency_f['이자보상배율'].fillna(0), -10, 10) / 10)
    if 'ROA' in df.columns:
        financial_health_components.append(np.clip(df['ROA'].fillna(0), -100, 100) / 100)
    
    if financial_health_components:
        features['재무건전성지수'] = np.mean(financial_health_components, axis=0)
    
    # 2. 유동성 스트레스 지수
    liquidity_stress_components = []
    
    if '운전자본비율' in liquidity_f.columns:
        liquidity_stress_components.append(-liquidity_f['운전자본비율'].fillna(0))
    if '단기부채비중' in liquidity_f.columns:
        liquidity_stress_components.append(liquidity_f['단기부채비중'].fillna(0))
    
    if liquidity_stress_components:
        features['유동성스트레스지수'] = np.mean(liquidity_stress_components, axis=0)
    
    # 3. 지급불능 위험 지수
    insolvency_risk_components = []
    
    if '자본잠식도' in insolvency_f.columns:
        insolvency_risk_components.append(insolvency_f['자본잠식도'].fillna(0))
    if '부채상환년수' in insolvency_f.columns:
        insolvency_risk_components.append(np.clip(insolvency_f['부채상환년수'].fillna(0), 0, 20) / 20)
    
    if insolvency_risk_components:
        features['지급불능위험지수'] = np.mean(insolvency_risk_components, axis=0)
    
    # 4. 시장 포지션 지수
    if '기업규모_로그' in korean_f.columns:
        size_normalized = (korean_f['기업규모_로그'] - korean_f['기업규모_로그'].mean()) / korean_f['기업규모_로그'].std()
        features['시장포지션지수'] = size_normalized.fillna(0)
    
    # 5. 종합 부도 위험 스코어
    risk_components = [
        features.get('유동성스트레스지수', 0) * 0.3,
        features.get('지급불능위험지수', 0) * 0.3,
        manipulation_f.get('재무조작위험', 0) * 0.1,
        korean_f.get('중소기업리스크', 0) * 0.1,
        stakeholder_f.get('이해관계자_불신지수', 0) / 10 * 0.2
    ]
    
    features['종합부도위험스코어'] = np.sum(risk_components, axis=0)
    
    # 6. 조기경보 시그널
    warning_signals = []
    
    if '자본잠식여부' in insolvency_f.columns:
        warning_signals.append(insolvency_f['자본잠식여부'])
    if '연체여부' in stakeholder_f.columns:
        warning_signals.append(stakeholder_f['연체여부'])
    if '현금소진일수' in liquidity_f.columns:
        warning_signals.append((liquidity_f['현금소진일수'] < 30).astype(int))
    
    if warning_signals:
        features['조기경보신호수'] = np.sum(warning_signals, axis=0)
        features['위험경보등급'] = pd.cut(features['조기경보신호수'], 
                                      bins=[-0.1, 0, 1, 2, 10],
                                      labels=['정상', '주의', '경고', '위험'])
    
    print(f"✅ 복합 리스크 지표 {features.shape[1]}개 생성 완료")
    return features

composite_features = create_composite_risk_indices(
    df, liquidity_features, insolvency_features, 
    manipulation_features, korean_features, stakeholder_features
)

✅ 복합 리스크 지표 7개 생성 완료


## 8. 비선형 관계 및 상호작용 특성

In [13]:
def create_interaction_features(df, features_dict):
    """주요 변수 간 상호작용 효과를 포착하는 특성 생성"""
    
    interactions = pd.DataFrame(index=df.index)
    
    # 1. 레버리지 × 수익성 상호작용
    if '부채비율' in df.columns and 'ROE' in df.columns:
        interactions['레버리지_수익성_상호작용'] = (df['부채비율'] / 100) * (df['ROE'] / 100)
        interactions['고레버리지_저수익'] = ((df['부채비율'] > 200) & (df['ROE'] < 0)).astype(int)
    
    # 2. 유동성 × 성장 상호작용
    if '유동비율' in df.columns and '매출액' in df.columns:
        매출성장률 = df['매출액'] / df['매출액'].median() - 1
        interactions['유동성_성장_상호작용'] = df['유동비율'] * 매출성장률
    
    # 3. 규모 × 효율성 상호작용
    if '기업규모_로그' in features_dict['korean'].columns and '총자산회전율' in df.columns:
        interactions['규모_효율성_상호작용'] = features_dict['korean']['기업규모_로그'] * df['총자산회전율']
    
    # 4. 업종 × 재무비율 상호작용
    if '제조업_레버리지위험' in features_dict['korean'].columns:
        interactions['제조업_부채위험'] = features_dict['korean']['제조업_레버리지위험']
    
    # 5. 비선형 변환
    if '부채비율' in df.columns:
        interactions['부채비율_제곱'] = (df['부채비율'] / 100) ** 2
        interactions['부채비율_로그'] = np.log1p(df['부채비율'].abs())
    
    if '현금비율' in df.columns:
        interactions['현금비율_제곱근'] = np.sqrt(df['현금비율'].abs())
    
    # 6. 임계값 기반 특성
    if '이자보상배율' in features_dict['insolvency'].columns:
        interactions['이자보상_임계'] = (features_dict['insolvency']['이자보상배율'] < 1).astype(int)
    
    if '운전자본' in features_dict['liquidity'].columns:
        interactions['운전자본_음수'] = (features_dict['liquidity']['운전자본'] < 0).astype(int)
    
    print(f"✅ 상호작용 및 비선형 특성 {interactions.shape[1]}개 생성 완료")
    return interactions

features_dict = {
    'liquidity': liquidity_features,
    'insolvency': insolvency_features,
    'manipulation': manipulation_features,
    'korean': korean_features,
    'stakeholder': stakeholder_features
}

interaction_features = create_interaction_features(df, features_dict)

✅ 상호작용 및 비선형 특성 3개 생성 완료


## 9. 모든 특성 통합

In [14]:
# 모든 특성을 하나의 데이터프레임으로 통합
all_features = pd.concat([
    liquidity_features,
    insolvency_features,
    manipulation_features,
    korean_features,
    stakeholder_features,
    composite_features,
    interaction_features
], axis=1)

print(f"\n✅ 총 {all_features.shape[1]}개의 도메인 기반 특성 생성 완료")
print("\n특성 카테고리별 개수:")
print(f"  - 유동성 위기: {liquidity_features.shape[1]}개")
print(f"  - 지급불능 패턴: {insolvency_features.shape[1]}개")
print(f"  - 재무조작 탐지: {manipulation_features.shape[1]}개")
print(f"  - 한국 시장 특화: {korean_features.shape[1]}개")
print(f"  - 이해관계자 행동: {stakeholder_features.shape[1]}개")
print(f"  - 복합 리스크 지표: {composite_features.shape[1]}개")
print(f"  - 상호작용/비선형: {interaction_features.shape[1]}개")

# 원본 데이터의 기본 정보와 타겟 변수 추가
final_features = pd.concat([
    df[target_col],
    all_features
], axis=1)

# 결측치 비율 확인
missing_ratio = final_features.isnull().mean() * 100
print(f"\n결측치 50% 이상인 특성 수: {(missing_ratio > 50).sum()}개")


✅ 총 65개의 도메인 기반 특성 생성 완료

특성 카테고리별 개수:
  - 유동성 위기: 10개
  - 지급불능 패턴: 8개
  - 재무조작 탐지: 15개
  - 한국 시장 특화: 13개
  - 이해관계자 행동: 9개
  - 복합 리스크 지표: 7개
  - 상호작용/비선형: 3개

결측치 50% 이상인 특성 수: 0개


## 10. 특성 중요도 초기 평가

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score

# 타겟과 특성 분리
X = all_features.copy()
y = df[target_col]

# -------------------------------------------------------------------------
# [수정된 부분] 범주형 변수 처리 (에러 해결 핵심)
# '위험경보등급'이 범주형(Category)이라서 median 계산 및 모델 학습 시 에러 발생
# -> 숫자로 변환 (정상:0, 주의:1, 경고:2, 위험:3)
# -------------------------------------------------------------------------
if '위험경보등급' in X.columns:
    # cat.codes는 -1(NaN), 0, 1, 2... 로 변환됨
    X['위험경보등급'] = X['위험경보등급'].cat.codes
    # 혹시 모를 -1(NaN) 값을 0으로 처리하거나 적절히 대체
    X['위험경보등급'] = X['위험경보등급'].replace(-1, 0)

# 결측치 처리 (중앙값으로 대체)
# 이제 모든 변수가 숫자형이므로 에러가 발생하지 않음
X_filled = X.fillna(X.median())

# 무한대 값 처리
X_filled = X_filled.replace([np.inf, -np.inf], 0)

# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_filled, y, test_size=0.2, random_state=42, stratify=y
)

# 간단한 Random Forest로 특성 중요도 평가
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf.fit(X_train, y_train)

# 특성 중요도 추출
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("\n🎯 상위 20개 중요 특성 (Random Forest 기준)")
print("="*60)
print(feature_importance.head(20).to_string(index=False))

# 시각화
fig = go.Figure()
fig.add_trace(go.Bar(
    y=feature_importance.head(20)['feature'].values[::-1],
    x=feature_importance.head(20)['importance'].values[::-1],
    orientation='h',
    marker_color='lightblue'
))

fig.update_layout(
    title='도메인 기반 특성 중요도 (상위 20개)',
    xaxis_title='Feature Importance',
    yaxis_title='특성명',
    height=600
)
fig.show()

# 간단한 성능 평가
y_pred_proba = rf.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
pr_auc = average_precision_score(y_test, y_pred_proba)

print(f"\n📊 초기 모델 성능 (도메인 특성만 사용)")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"PR-AUC: {pr_auc:.4f}")


🎯 상위 20개 중요 특성 (Random Forest 기준)
   feature  importance
이해관계자_불신지수    0.115817
    신용등급점수    0.083416
     연체심각도    0.033175
   조기경보신호수    0.029903
     총연체건수    0.028992
 종합부도위험스코어    0.027431
      연체여부    0.025684
    재무레버리지    0.024257
    부채레버리지    0.023730
    부채상환년수    0.020146
    매출총이익률    0.020110
     순부채비율    0.019389
     이자부담률    0.019175
   기업규모_로그    0.019031
      판관비율    0.018474
      운전자본    0.018116
   시장포지션지수    0.018044
    단기부채비중    0.017896
   유동성압박지수    0.017864
    이자보상배율    0.017796



📊 초기 모델 성능 (도메인 특성만 사용)
ROC-AUC: 0.8235
PR-AUC: 0.1151


## 11. 특성 저장

In [17]:
# 생성된 특성을 CSV 파일로 저장
output_path = '../data/features/domain_based_features.csv'
final_features.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n✅ 도메인 기반 특성이 {output_path}에 저장되었습니다.")

# 특성 메타데이터 저장
feature_metadata = pd.DataFrame({
    'category': (
        ['liquidity'] * liquidity_features.shape[1] +
        ['insolvency'] * insolvency_features.shape[1] +
        ['manipulation'] * manipulation_features.shape[1] +
        ['korean'] * korean_features.shape[1] +
        ['stakeholder'] * stakeholder_features.shape[1] +
        ['composite'] * composite_features.shape[1] +
        ['interaction'] * interaction_features.shape[1]
    ),
    'feature_name': all_features.columns.tolist(),
    'importance': feature_importance.set_index('feature')['importance'].reindex(all_features.columns).values
})

metadata_path = '../data/features/feature_metadata.csv'
feature_metadata.to_csv(metadata_path, index=False, encoding='utf-8-sig')
print(f"✅ 특성 메타데이터가 {metadata_path}에 저장되었습니다.")


✅ 도메인 기반 특성이 ../data/features/domain_based_features.csv에 저장되었습니다.
✅ 특성 메타데이터가 ../data/features/feature_metadata.csv에 저장되었습니다.


## 12. 핵심 인사이트 정리

In [18]:
print("\n" + "="*80)
print("📌 도메인 기반 특성 공학 - 핵심 인사이트")
print("="*80)

insights = [
    "1. 유동성 위기 조기 감지: 현금소진일수, 운전자본 건전성 등 즉각적 지급능력 특성 생성",
    "2. 지급불능 패턴: 자본잠식도, 부채상환년수 등 장기 지급능력 평가 지표 구현",
    "3. 재무조작 탐지: 한국형 M-Score를 통한 회계 이상 징후 포착",
    "4. 한국 시장 특화: 대기업 의존도, 제조업 특성, 외감 여부 등 한국 기업 특성 반영",
    "5. 이해관계자 신뢰: 연체, 세금체납, 신용등급 등 외부 신뢰도 지표 통합",
    "6. 복합 리스크 지표: 다양한 도메인 지식을 통합한 종합 위험 스코어 개발",
    "7. 비선형 관계: 제곱, 로그, 상호작용 항을 통한 복잡한 패턴 포착"
]

for insight in insights:
    print(insight)

print("\n" + "="*80)
print("🎯 다음 단계 권장사항")
print("="*80)

recommendations = [
    "1. 상관관계 분석을 통한 다중공선성 제거",
    "2. Information Value 계산을 통한 특성 선택",
    "3. SMOTE + Tomek Links 적용 전 특성 스케일링",
    "4. 카테고리별 특성 중요도 기반 가중 앙상블 모델 구축",
    "5. SHAP 분석을 통한 특성 기여도 해석"
]

for rec in recommendations:
    print(rec)


📌 도메인 기반 특성 공학 - 핵심 인사이트
1. 유동성 위기 조기 감지: 현금소진일수, 운전자본 건전성 등 즉각적 지급능력 특성 생성
2. 지급불능 패턴: 자본잠식도, 부채상환년수 등 장기 지급능력 평가 지표 구현
3. 재무조작 탐지: 한국형 M-Score를 통한 회계 이상 징후 포착
4. 한국 시장 특화: 대기업 의존도, 제조업 특성, 외감 여부 등 한국 기업 특성 반영
5. 이해관계자 신뢰: 연체, 세금체납, 신용등급 등 외부 신뢰도 지표 통합
6. 복합 리스크 지표: 다양한 도메인 지식을 통합한 종합 위험 스코어 개발
7. 비선형 관계: 제곱, 로그, 상호작용 항을 통한 복잡한 패턴 포착

🎯 다음 단계 권장사항
1. 상관관계 분석을 통한 다중공선성 제거
2. Information Value 계산을 통한 특성 선택
3. SMOTE + Tomek Links 적용 전 특성 스케일링
4. 카테고리별 특성 중요도 기반 가중 앙상블 모델 구축
5. SHAP 분석을 통한 특성 기여도 해석
